In [23]:
import sys

sys.path.insert(1, '../')

In [24]:
from scripts.search import get_supplier

In [2]:
import openai
from openai import OpenAI

openai_api_key = 'sk-proj-QasBxGfGAE328TSt1cuzT3BlbkFJllWJJz5VygPxStQMhsMf'
openai.api_key = openai_api_key

In [73]:
history = [
    {
        "role": "assistant", 
        "content": "Halo, apa yang bisa saya bantu?"
    },
    # {
    #     "role": "user",
    #     "content": "Saya sedang mencari supplier untuk usaha furniture di sekitar Bogor atau Tanggerang. Ada rekomendasi nggak?"
    # },
    {
        "role": "user",
        "content": "Cara membuat nasi goreng?"
    },
]

In [74]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_supplier",
            "description": "Mendapatkan informasi supplier",
            "parameters": {
                "type": "object",
                "properties": {
                    "barang_jasa": {
                        "type": "string",
                        "description": "Barang atau jasa yang ingin di cari",
                    },
                    "provinsi": {
                        "type": "array",
                        "description": "Daftar nama provinsi, contoh: Jawa Barat",
                        "items": {
                            "type": "string"
                        }
                    },
                    "kota": {
                        "type": "array",
                        "description": "Daftar nama kota, contoh: Bogor",
                        "items": {
                            "type": "string"
                        }
                    },
                    "kecamatan": {
                        "type": "array",
                        "description": "Daftar nama kecamatan, contoh: Bogor Timur",
                        "items": {
                            "type": "string"
                        }
                    },
                },
                "required": ["barang_jasa"],
            },
        },
    }
]

In [75]:
client = OpenAI(api_key=openai_api_key)
response = client.chat.completions.create(
    model="gpt-3.5-turbo", 
    messages=history,
    tools=tools,
    tool_choice="auto",  # auto is default, but we'll be explicit
)
response

ChatCompletion(id='chatcmpl-9gCllZp55rwSqmQdMWwhcl8bHiPrI', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Berikut ini adalah langkah-langkah membuat nasi goreng:\n\nBahan-bahan:\n1. Nasi putih (bekas atau dingin)\n2. Bawang putih, bawang merah, dan cabai merah (sesuai selera)\n3. Daging ayam, daging sapi, atau udang (opsional)\n4. Telur\n5. Kecap manis\n6. Garam, gula, lada, dan penyedap rasa\n\nLangkah-langkah:\n1. Tumis bawang putih, bawang merah, dan cabai merah hingga harum.\n2. Tambahkan daging ayam, daging sapi, atau udang. Aduk hingga matang.\n3. Masukkan telur dan aduk rata hingga telur matang.\n4. Masukkan nasi putih dan aduk hingga tercampur rata.\n5. Tambahkan kecap manis, garam, gula, lada, dan penyedap rasa sesuai selera.\n6. Aduk terus hingga semua bahan tercampur dan matang.\n7. Sajikan nasi goreng panas dengan irisan telur mata sapi dan acar timun.\n\nSelamat mencoba membuat nasi goreng di rumah! Jika Anda membutuhk

In [76]:
response_message = response.choices[0].message
tool_calls = response_message.tool_calls[0]

response_message

TypeError: 'NoneType' object is not subscriptable

In [79]:
response_message.content

'Berikut ini adalah langkah-langkah membuat nasi goreng:\n\nBahan-bahan:\n1. Nasi putih (bekas atau dingin)\n2. Bawang putih, bawang merah, dan cabai merah (sesuai selera)\n3. Daging ayam, daging sapi, atau udang (opsional)\n4. Telur\n5. Kecap manis\n6. Garam, gula, lada, dan penyedap rasa\n\nLangkah-langkah:\n1. Tumis bawang putih, bawang merah, dan cabai merah hingga harum.\n2. Tambahkan daging ayam, daging sapi, atau udang. Aduk hingga matang.\n3. Masukkan telur dan aduk rata hingga telur matang.\n4. Masukkan nasi putih dan aduk hingga tercampur rata.\n5. Tambahkan kecap manis, garam, gula, lada, dan penyedap rasa sesuai selera.\n6. Aduk terus hingga semua bahan tercampur dan matang.\n7. Sajikan nasi goreng panas dengan irisan telur mata sapi dan acar timun.\n\nSelamat mencoba membuat nasi goreng di rumah! Jika Anda membutuhkan bantuan lain, jangan ragu untuk bertanya.'

In [57]:
tool_calls

ChatCompletionMessageToolCall(id='call_3f01wWWaVIMwBhGe6xn09piF', function=Function(arguments='{"barang_jasa":"furniture","kota":["Bogor","Tangerang"]}', name='get_supplier'), type='function')

In [58]:
tool_calls.function

Function(arguments='{"barang_jasa":"furniture","kota":["Bogor","Tangerang"]}', name='get_supplier')

In [59]:
(tool_calls.function.name, tool_calls.function.arguments)

('get_supplier', '{"barang_jasa":"furniture","kota":["Bogor","Tangerang"]}')

In [60]:
import json

args = json.loads(tool_calls.function.arguments)
args

{'barang_jasa': 'furniture', 'kota': ['Bogor', 'Tangerang']}

In [61]:
barang_jasa = args['barang_jasa']
provinsi = args.get('provinsi', [])
kota = args.get('kota', [])
kecamatan = args.get('kecamatan', [])

(barang_jasa, provinsi, kota, kecamatan)

('furniture', [], ['Bogor', 'Tangerang'], [])

In [62]:
if tool_calls.function.name == 'get_supplier':
    df = get_supplier(barang_jasa, provinsi=provinsi, kota=kota, kecamatan=kecamatan)
    df

filter_str: (kota eq 'Bogor' or kota eq 'Tangerang')


In [63]:
df

,id,bidang,supplier,deskripsi,barang,provinsi,kota,kecamatan
0,93,Furniture,PT Lumber Supplies,Supplier kayu untuk pembuatan furniture,"Kayu Pinus, Kayu Meranti",Banten,Tangerang,Karawaci
1,99,Furniture,PT Hardwood Solutions,Supplier kayu keras,"Kayu Pinus, Kayu Meranti",Jawa Barat,Bogor,Bogor Timur


In [64]:
supplier_content = df.to_markdown()
print(supplier_content)

|    |   id | bidang    | supplier              | deskripsi                               | barang                   | provinsi   | kota      | kecamatan   |
|---:|-----:|:----------|:----------------------|:----------------------------------------|:-------------------------|:-----------|:----------|:------------|
|  0 |   93 | Furniture | PT Lumber Supplies    | Supplier kayu untuk pembuatan furniture | Kayu Pinus, Kayu Meranti | Banten     | Tangerang | Karawaci    |
|  1 |   99 | Furniture | PT Hardwood Solutions | Supplier kayu keras                     | Kayu Pinus, Kayu Meranti | Jawa Barat | Bogor     | Bogor Timur |


# Final Answer

In [67]:
history2 = list(history)
history2.append({
    'role': 'system',
    'content': f'Daftar UMKM yang tersedia:\n{supplier_content}'
})
print(json.dumps(history2, indent=4))

[
    {
        "role": "assistant",
        "content": "Halo, apa yang bisa saya bantu?"
    },
    {
        "role": "user",
        "content": "Saya sedang mencari supplier untuk usaha furniture di sekitar Bogor atau Tanggerang. Ada rekomendasi nggak?"
    },
    {
        "role": "system",
        "content": "Daftar UMKM yang tersedia:\n|    |   id | bidang    | supplier              | deskripsi                               | barang                   | provinsi   | kota      | kecamatan   |\n|---:|-----:|:----------|:----------------------|:----------------------------------------|:-------------------------|:-----------|:----------|:------------|\n|  0 |   93 | Furniture | PT Lumber Supplies    | Supplier kayu untuk pembuatan furniture | Kayu Pinus, Kayu Meranti | Banten     | Tangerang | Karawaci    |\n|  1 |   99 | Furniture | PT Hardwood Solutions | Supplier kayu keras                     | Kayu Pinus, Kayu Meranti | Jawa Barat | Bogor     | Bogor Timur |"
    }
]


In [68]:
client = OpenAI(api_key=openai_api_key)
response = client.chat.completions.create(
    model="gpt-3.5-turbo", 
    messages=history2,
)
response

ChatCompletion(id='chatcmpl-9gCiQtABH0PA3awJEckQuQAIAZNnL', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Berikut adalah daftar supplier furniture di sekitar Bogor dan Tangerang:\n1. PT Lumber Supplies - Tangerang\n   - Deskripsi: Supplier kayu untuk pembuatan furniture\n   - Barang: Kayu Pinus, Kayu Meranti\n   - Lokasi: Tangerang, Karawaci\n\n2. PT Hardwood Solutions - Bogor\n   - Deskripsi: Supplier kayu keras\n   - Barang: Kayu Pinus, Kayu Meranti\n   - Lokasi: Bogor, Bogor Timur\n\nAnda bisa menghubungi kedua supplier di atas untuk informasi lebih lanjut atau pertanyaan tentang kebutuhan furniture Anda. Semoga membantu!', role='assistant', function_call=None, tool_calls=None))], created=1719845682, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=151, prompt_tokens=223, total_tokens=374))

In [72]:
print(response.choices[0].message.content)

Berikut adalah daftar supplier furniture di sekitar Bogor dan Tangerang:
1. PT Lumber Supplies - Tangerang
   - Deskripsi: Supplier kayu untuk pembuatan furniture
   - Barang: Kayu Pinus, Kayu Meranti
   - Lokasi: Tangerang, Karawaci

2. PT Hardwood Solutions - Bogor
   - Deskripsi: Supplier kayu keras
   - Barang: Kayu Pinus, Kayu Meranti
   - Lokasi: Bogor, Bogor Timur

Anda bisa menghubungi kedua supplier di atas untuk informasi lebih lanjut atau pertanyaan tentang kebutuhan furniture Anda. Semoga membantu!
